In [1]:
# Fusion ART Model for Professor Data
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

import numpy as np

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [2]:
# Load a session
dill.load_session('notebook_env_rep.db')

In [3]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [4]:
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text

In [5]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [6]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [7]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_processed = []
for entry in university:
    uni = entry.split(";")
    university_processed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_preprocessed to 1D lists for one hot encoding
university_1d = []
for universities in university_processed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        


In [8]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_processed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))


In [9]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [10]:
# One Hot Encoding for name, group, university and research interests
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)

research_interest_integer_encoded = research_label_encoder.fit_transform(array(research_interest_1d))
research_interest_integer_encoded = research_interest_integer_encoded.reshape(len(research_interest_integer_encoded), 1)
research_interest_onehot = onehot_encoder.fit_transform(research_interest_integer_encoded)


In [11]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()
research_interest_onehotlist = research_interest_onehot.tolist()


In [12]:
# Perform the one-hot research keywords noise flipping here

def one_hot_keyword_flip(query_research, noise_limit):
    
    query_research = query_research.split(";")
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        for onehot_vec in research_interest_onehotlist:
            if (str(query_research[research_index]).rstrip("\n") == 
                research_label_encoder.inverse_transform([argmax(onehot_vec)])[0]):
                query_research[research_index] = onehot_vec
                

        
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1


    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            if (research_vector[index] == 0):
                research_vector[index] = 1
            else:
                research_vector[index] = 0
    return research_vector


In [15]:
# Research keywords flip for 0% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_zero_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [14]:
# Research keywords flip for 10% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.1)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_ten_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)


In [153]:
# Research keywords flip for 20% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.2)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_twenty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [154]:
# Research keywords flip for 30% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.3)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_thirty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [155]:
# Research keywords flip for 40% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.4)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_forty_research_query.txt", "w+")
#oisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [156]:
# Research keywords flip for 50% noise
name_keyword_dict = {}

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        flipped_vector = one_hot_keyword_flip(keywords, 0.5)
        name_keyword_dict[batch_name] = flipped_vector
        
        
# Create the noisy binary keyword vector and create the new input text file from the noisy binary keyword vector
output_file = open("noise_fifty_research_query.txt", "w+")
noisy_research_interest_1d = []

noisy_research_interest_per_person = []
for professor in name:
    num_res_ints = 0
    noisy_research_vector = name_keyword_dict[professor]
    output_file.write(professor+"\t")
    for index in range(0, len(noisy_research_vector)):
        # Split the combined research keyword vectors into individual vectors
        if noisy_research_vector[index] == 1.0:
            ind_list = [0] * len(noisy_research_vector)
            ind_list[index] = 1
            num_res_ints += 1
            research_name = research_label_encoder.inverse_transform([argmax(ind_list)])[0]
            noisy_research_interest_1d.append(research_name)
            output_file.write(research_name+";")
    output_file.write("\n")
    noisy_research_interest_per_person.append(num_res_ints)

In [45]:
"""
# Create the new input text file from the noisy binary keyword vector
output_file = open("noise_ten_research_query.txt", "w+")
index = 0
first = 1
for professor in name:
    output_file.write(professor+"\t")
    for keyword in range(sum(noisy_research_interest_per_person[0:index]), sum(noisy_research_interest_per_person[0:index+1])):
        output_file.write(noisy_research_interest_1d[keyword])
        if keyword != (sum(noisy_research_interest_per_person[:index+1]) - 1):
            output_file.write(";")
        
    index += 1
    output_file.write("\n")
"""

In [21]:
# Create a list of unique names, groups, universities and research interests to define the model schema
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass

In [22]:
# Create a word2vec model that contains all research keywords
research_tokens = convert_to_2d(research_interest_1d, research_interest_per_person)
research_word2vec = Word2Vec(research_tokens, min_count = 0)


In [23]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass


C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [24]:
# Clean all text from Wikipedia articles
for text_index in range(0, len(wiki_text)):
    for word_index in range(0, len(wiki_text[text_index])):
        wiki_text[text_index][word_index] = clean_text(wiki_text[text_index][word_index])
        

In [25]:
# Search for research keyword n-grams from Wikipedia articles
ngram_wiki_tokens = []

for text in wiki_text:
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest, text))

In [26]:
# Train the word2vec model from the cleaned, n-grammed Wikipedia tokens
research_word2vec.build_vocab(ngram_wiki_tokens, update = True)
research_word2vec.train(ngram_wiki_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(4523520, 5708255)

In [27]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('testtext.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [28]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []
for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest, cleaned_text))

In [29]:
# Train the word2vec model by including the cleaned, n-grammed custom corpus tokens
research_word2vec.build_vocab(ngram_test_tokens, update = True)
research_word2vec.train(ngram_test_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(168726, 219210)

In [36]:
# Vocabulary list for the word2vec model
list(research_word2vec.wv.vocab)

['computer_graphics',
 'shape_modelling',
 'virtual_reality',
 'web_visualization',
 'visualization_on_the_grid',
 'cybermedicine',
 'scientific_visualization',
 'high_level_synthesis',
 'application_specific_processors',
 'heterogeneous_mpsoc',
 'synthesis_for_emerging_technology',
 'distributed_systems',
 'security_and_privacy',
 'data_analytics',
 'algorithms',
 'self_organization',
 'graphs_querying_and_mining',
 'databases',
 'data_mining',
 'machine_learning',
 'real_time_systems',
 'cyber_physical_systems',
 'formal_methods',
 'ultra_wideband_radio',
 'modulation_and_multiple_access',
 'communication_algorithms',
 'digital_signal_processing',
 'multi_agent_systems',
 'game_theory',
 'optimization',
 'intelligent__e-commerce',
 'image_coding',
 'video_coding',
 'wireless_video',
 'multimedia_networking',
 'parallel_and_distributed_simulation',
 'programming_environments_and_tools',
 'parallel_algorithms_and_architectures',
 'system_performance_analysis',
 'computer_vision',
 'int

In [31]:
# Find the top 5 most similar keywords for all research keywords
for keyword in sorted(unique_research_interest):
    print("Top 5 similar words from "+keyword+": ")
    print(research_word2vec.wv.most_similar(keyword, topn = 5))
    print()

Top 5 similar words from ad_hoc_and_mobile_networks: 
[('rationalize', 0.3104821741580963), ('characteristicfeatures', 0.2885429859161377), ('workbench', 0.28667664527893066), ('logiccoq', 0.2822013199329376), ('gamesin', 0.2780315577983856)]

Top 5 similar words from agent_oriented_software_engineering: 
[('software_engineering', 0.8574602007865906), ('whitestein', 0.8569512963294983), ('toolkits', 0.8381093740463257), ('gnu', 0.8332349061965942), ('netowl', 0.8323467969894409)]

Top 5 similar words from algorithms: 
[('techniques', 0.8491071462631226), ('methods', 0.8343086838722229), ('models', 0.8233515024185181), ('problems', 0.815708577632904), ('approaches', 0.7955188155174255)]

Top 5 similar words from algorithms_and_data_structure: 
[('mathworld', 0.762791633605957), ('econometric', 0.7245378494262695), ('envelopment', 0.7074993848800659), ('tekken', 0.6922211647033691), ('ignorance', 0.6845632791519165)]

Top 5 similar words from animation_and_visualization: 
[('ergodic', 0.

[('freedoms', 0.31030893325805664), ('“quartile”', 0.29226356744766235), ('didnt', 0.2859705090522766), ('latters', 0.28181254863739014), ('cagd_and_geometric_modeling', 0.27319440245628357)]

Top 5 similar words from distributed_machine_learning: 
[('asymptote', 0.33659327030181885), ('0s', 0.30831247568130493), ('perceptional', 0.305833101272583), ('unresolved', 0.30481114983558655), ('correlates', 0.3014346957206726)]

Top 5 similar words from distributed_systems: 
[('recommender_systems', 0.8807242512702942), ('recommender', 0.8788201808929443), ('cyberphysical', 0.8706617951393127), ('synchronizers', 0.8680647611618042), ('hybrid', 0.8597177863121033)]

Top 5 similar words from distributed_systems_security: 
[('fragmented', 0.4252205491065979), ('demonstrating', 0.3820025324821472), ('colocated', 0.3819330930709839), ('madethe', 0.3778536319732666), ('‘sexy’’’', 0.37067580223083496)]

Top 5 similar words from distributed_virtual_environments: 
[('eta', 0.32253748178482056), ('mult

[('gmmhmm', 0.6326992511749268), ('preferencea', 0.6317993402481079), ('substructure', 0.6264601945877075), ('checking', 0.6097586750984192), ('kafka', 0.6089937686920166)]

Top 5 similar words from model_checking: 
[('checking', 0.9838183522224426), ('downsian', 0.8826970458030701), ('checker', 0.8804929256439209), ('signalflow', 0.8800953030586243), ('calculi', 0.8755069971084595)]

Top 5 similar words from modelling_of_manipulative_interaction: 
[('computationsalternatively', 0.318020761013031), ('woelfel', 0.3074544370174408), ('gil', 0.2849401831626892), ('surprise', 0.2779882550239563), ('mcdonough', 0.2725571393966675)]

Top 5 similar words from modulation_and_multiple_access: 
[('sole', 0.34276968240737915), ('ipem', 0.3325226306915283), ('software_security', 0.32588109374046326), ('exaggerated', 0.29347360134124756), ('surfacing', 0.28484588861465454)]

Top 5 similar words from multi_agent_system: 
[('ids', 0.8914415836334229), ('onehybrid', 0.883908748626709), ('fais', 0.8823

[('doublespending', 0.2837081551551819), ('enact', 0.2817591428756714), ('dopaminebased', 0.2766348123550415), ('asynchronicity', 0.2509009838104248), ('distributional', 0.24464674293994904)]

Top 5 similar words from text_mining: 
[('data_mining', 0.9070155024528503), ('wikipediadata', 0.8806748390197754), ('synergy', 0.8748980760574341), ('mining', 0.8673481345176697), ('big_data_analytics', 0.8536703586578369)]

Top 5 similar words from tool_condition_monitoring: 
[('condition', 0.8466603755950928), ('dfs', 0.819360077381134), ('sentencepattern', 0.810140073299408), ('forensic', 0.8087533712387085), ('tcm', 0.8035500049591064)]

Top 5 similar words from transfer_learning: 
[('multitask', 0.8951634168624878), ('semisupervised', 0.8827099204063416), ('federated', 0.8818086981773376), ('lvq', 0.8617768287658691), ('incremental', 0.8611868619918823)]

Top 5 similar words from trust_management: 
[('trust', 0.9079002141952515), ('cooperation', 0.8272837400436401), ('workflow', 0.826538562

In [32]:
"""
SIMILARITY
# Map each research keyword as a weight vector that measures its similarity with all other research keywords
research_interest_weight_dict = defaultdict(list)
for keyword1 in sorted(unique_research_interest):
    keyword1_weights = []
    for keyword2 in sorted(unique_research_interest):
        # Round to 5 decimal places for all similarity scores
        keyword1_weights.append(round(research_word2vec.wv.similarity(w1=keyword1, w2=keyword2), 5))
    research_interest_weight_dict[keyword1] = keyword1_weights
    
# Provide weight vectors for each research interest
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_interest_weight_dict[res_int])
"""

In [54]:
"""
WORD EMBEDDING
# Provide weight vectors for each research interest
"""
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_word2vec[res_int])

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [55]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_weights, research_interest_per_person)

In [84]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []

for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])

    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)]

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [85]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))

In [19]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [20]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [21]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

In [22]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")
    
    
    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")
    
    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research) - 1):
        # WORD EMBEDDING
        query_research[research_index] = research_word2vec[query_research[research_index]].tolist()
        
        # SIMILARITY
        # query_research[research_index] = research_interest_weight_dict[query_research[research_index]]
        
        
    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if (type(query_research[index]) != list or query_research[index] == []):
            del query_research[index]
        else:
            index += 1
    

    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]            
        
    
    # Normalise the vector input to a range between 0 and 1
    normalised_research_vector = normalise_vector(research_vector)
    
    # Define the complement-coded research vector
    compl_vector = []
    for bit in normalised_research_vector:
        compl_vector.append(1 - bit)
        
    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector, 'vcompl': compl_vector}], refresh = False)
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [23]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_zero_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1
        
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [158]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_ten_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [159]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_twenty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [160]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_thirty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [161]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_forty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [162]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('noise_fifty_research_query.txt', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file)
        if (retrieved_name == batch_name):
            correct_match += 1

query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
